In [1]:
import pickle
from pathlib import Path
from math import exp, log

In [2]:
# Define the classes

def classes(code):

    high = code[0]

    try:
        low = int(code[1])
    except IndexError:
        low = ""

    if high == "p":
        if low == "":
            return "Political"
        elif low == 1:
            return "War"
        elif low == 2:
            return "Dividedness"
        elif low == 3:
            return "Scandals"
        elif low == 4:
            return "Cultural diplomacy"
        else:
            return "Incorrect code"
    elif high == "e":
        if low == "":
            return "Economic"
        elif low == 1:
            return "Economic instability"
        elif low == 2:
            return "Government policy"
        elif low == 3:
            return "Economic Stagnation"
        else:
            return "Incorrect code"
    else:
        return "Incorrect code"

In [3]:
NamedCountries = ["Singapore", "UK", "India", "US"]

In [4]:
# Create a node: a class that has the behaviour of a node

class node:
    def __init__(self, country, subclass):
        self.country = country
        self.subclass = subclass

        self.value = 0 # No risk when initialised

        self.events = [] # Store the ids of events that have affected this node

        self.change = 0
    
    def update(self, score):
        self.change = score

        score += self.value

        def UpdateFunction(score):
            return - exp(-(score/5 - log(10))) + 10
        
        self.value = UpdateFunction(score)

In [5]:
# Create a class to store bilateral risks

class bilateral:
    def __init__(self, countries):
        self.cause = [] # Refer to an event ID

        self.countries = [] # Countries is a list of length 2 containing class instances of 2 countries

        for country in countries:
            self.countries.append(country.name)

        self.political = node(self.countries, classes("p"))
        self.economic = node(self.countries, classes("e"))

        # Weight gives a link from the bilateral relations to the weights of the first country
        self.weights = [[0.5 for i in range(7)] for j in range(2)]

        self.loc = "Weights\\" + "".join(self.countries) + ".bin"
        self.path = Path(self.loc)

        if self.path.is_file():
            with open(self.loc, "rb") as f:
                self.weights = pickle.load(f)
        else:
            self.weights = {"domestic": [[0.5 for i in range(7)] for j in range(2)]}
            with open(self.loc, "wb") as f:
                pickle.dump(self.weights, f)

Order of weights:
1. Domestic: 7 x 7 where 1st index is start and 2nd index is end. Order is p1, p2, p3, p4, e1, e2, e3
2. < Country name >: 7 x 9  each index in the 2 d matrix gives a connection and the direction is from this country to the country it is connected to (the other direction is accessible from the other country)
    **The last two in the 9 are the bilateral nodes.**
3. Bilateral comes at the end as a set of 2: order is Political = 0 then Economic = 1

In [6]:
def GetBilateral(country1id, country2id):
    # The input is the ids of two countries
    # The index of country1 in countries is less than the index of country2

    a = [i for i in range(len(countries))]

    loopno = 0

    for i in range(len(a)):
        for j in range(len(a) - i - 1):
            if [a[i], a[j + i + 1]] == [country1id, country2id]:
                return loopno
            loopno += 1
            
    print("This combination of countries is invalid")

    return BilateralInfo[loopno]
    
    # The output is the class instace of the bilateral relations between the countries in the list BilateralInfo

In [7]:
# Create a class for the different countries in the graph

class domestic:
    def __init__(self, name):
        self.name = name
        self.id = CountryID = NamedCountries.index(name)
        self.nodes = []

        # Political
        for i in range(1, 5):
            self.nodes.append(node(self.name, classes("p" + str(i))))

        # Economic
        for i in range(1, 4):
            self.nodes.append(node(self.name, classes("e" + str(i))))

        self.loc = "Weights\\" + self.name + ".bin"
        self.path = Path(self.loc)

        if self.path.is_file():
            with open(self.loc, "rb") as f:
                self.weights = pickle.load(f)
        else:
            self.weights = {"domestic": [[0.5 for i in range(7)] for j in range(7)]}
            with open(self.loc, "wb") as f:
                pickle.dump(self.weights, f)

    def AddLink(self, country):
        # Country is a class instance of the domestic class
        if self.name == country.name:
            return None

        self.weights[country.name] = [[0.5 for i in range(9)] for j in range(7)]
        
        # This function only creates weights in one direction
    
    def FindNode(self, code):
        if code[0] == "p":
            return int(code[1]) - 1
        elif code[0] == "e":
            return int(code[1]) + 4
        else:
            print("Invalid subclass code")
            return None
        
        # Thus function returns the node number in the pre-defined format

    def propagate(self, StartNode):
        # Only changes get propagrated, not actual values
        # Self.change has to be updated before this behaviour is called

        # Start node is the number of the node - an integer from 1 to 7

        # Domestic propagation
        for i in range(7):
            if i != StartNode:
                self.nodes[i].update(self.nodes[StartNode].change * self.weights["domestic"][StartNode][i])
                
                if self.nodes[i].value > 8:
                    print(f"There is a grave situation in {self.name} in the field of {self.nodes[i].subclass} due to an event {self.nodes[StartNode].events[-1]}.")
        
        # International propagation
        for country in self.weight.keys():
            if country != "domestic":
                CountryID = NamedCountries.index(country)

                for i in range(7):
                    countries[CountryID].nodes[i].update(self.nodes[StartNode].change * self.weights[country][StartNode][i])
                
                rel = GetBilateral()
                i = 7
                rel.political.update(self.nodes[StartNode].change * self.weights[country][StartNode][i])
                rel.cause.append(self.nodes[StartNode].events[-1])
                
                i = 8
                rel.economic.update(self.nodes[StartNode].change * self.weights[country][StartNode][i])
                rel.cause.append(self.nodes[StartNode].events[-1])

                if ( rel.political.value + rel.economic.value ) / 2 > 8:
                    print(f"There is a grave situation in the relations between {self.name} and {country} due to an event {self.nodes[StartNode].events[-1]}.")

In [9]:
countries = []

for name in NamedCountries:
    countries.append(domestic(name))

# Intialise bilateral relations

BilateralInfo = []

for i in range(len(countries)):
    for j in range(len(countries) - i - 1):
        BilateralInfo.append(bilateral([countries[i], countries[j + i + 1]]))

In [10]:
countries[3].weights

{'domestic': [[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
  [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
  [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
  [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
  [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
  [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
  [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]]}

# Process events

In [ ]:
def EventProcess(id, country, subclass, score):

    """"
    ID gives the ID of the event in the event database
    Country is the country where the event took place
    Subclass is the subclass of the event
    Score is the change in the risk score due to that event
    """
    CountryID = NamedCountries.index(country)
    NodeAffected = countries[CountryID].FindNode(subclass)

    countries[CountryID].nodes[NodeAffected].update(score)
    countries[CountryID].nodes[NodeAffected].events.append(id)

    # The next step is to propagate the change to the surroudning nodes

    countries[CountryID].propagate